<a href="https://colab.research.google.com/github/chicks2014/text_classification_transformers/blob/main/Text_classification_simpleTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drive setup**

In [ ]:
!pip install simpletransformers

In [ ]:
import os

ROOT= "/content/drive/MyDrive/DL/NLP"

In [ ]:
os.chdir(ROOT)

In [ ]:
os.getcwd()

'/content/drive/MyDrive/DL/NLP'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Import Dataset**

In [ ]:
# ! pip install -q kaggle

In [ ]:
# from google.colab import files

# files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"chicks002","key":"39cb79ee1e8e25f1e6c2521fa9e5b78a"}'}

In [ ]:
# !mkdir ~/.kaggle

In [ ]:
# !cp kaggle.json ~/.kaggle/

In [ ]:
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# !kaggle datasets list

ref                                                              title                                               size  lastUpdated          downloadCount  
---------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
prasertk/healthy-lifestyle-cities-report-2021                    Healthy Lifestyle Cities Report 2021                 2KB  2022-03-03 00:26:02           1089  
yasserh/wine-quality-dataset                                     Wine Quality Dataset                                21KB  2022-01-15 19:15:11           9561  
piterfm/2022-ukraine-russian-war                                 2022 Ukraine Russia War                             880B  2022-03-10 11:05:31            877  
majyhain/height-of-male-and-female-by-country-2022               Height of Male and Female by Country 2022            4KB  2022-02-02 00:40:19           4795  
prasertk/netflix-subscription-price-in-d

In [ ]:
# !kaggle datasets download -d datatattle/covid-19-nlp-text-classification

  0% 0.00/4.38M [00:00<?, ?B/s]
100% 4.38M/4.38M [00:00<00:00, 71.9MB/s]


In [ ]:
# !mkdir covid19_nlp_text_classification

In [ ]:
# !unzip covid-19-nlp-text-classification.zip -d covid19_nlp_text_classification

Archive:  covid-19-nlp-text-classification.zip
  inflating: covid19_nlp_text_classification/Corona_NLP_test.csv  
  inflating: covid19_nlp_text_classification/Corona_NLP_train.csv  


# **Text Classification**

In [ ]:
#import lib
from simpletransformers.classification import ClassificationModel
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [ ]:
train_data = pd.read_csv('covid19_nlp_text_classification/Corona_NLP_train.csv', encoding='latin-1')

In [ ]:
train_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
train_data['Sentiment']= label_encoder.fit_transform(train_data['Sentiment'])
 
train_data['Sentiment'].unique()

array([3, 4, 0, 2, 1])

In [ ]:
# neutral = 3 , positive = 4, Extremely Negative = 0

In [ ]:
train_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,3
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,4
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,4
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,4
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0


In [ ]:
train_data.drop(columns = ['UserName','ScreenName','Location','TweetAt'], inplace=True)

In [ ]:
train_data.head()

,OriginalTweet,Sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,3
1,advice Talk to your neighbours family to excha...,4
2,Coronavirus Australia: Woolworths to give elde...,4
3,My food stock is not the only one which is emp...,4
4,"Me, ready to go at supermarket during the #COV...",0


In [ ]:
train_data.rename(columns = {'OriginalTweet' : 'text' , 'Sentiment':'labels'})

,text,labels
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,3
1,advice Talk to your neighbours family to excha...,4
2,Coronavirus Australia: Woolworths to give elde...,4
3,My food stock is not the only one which is emp...,4
4,"Me, ready to go at supermarket during the #COV...",0
...,...,...
41152,Airline pilots offering to stock supermarket s...,3
41153,Response to complaint not provided citing COVI...,0
41154,You know itÂs getting tough when @KameronWild...,4
41155,Is it wrong that the smell of hand sanitizer i...,3


In [ ]:
eval_data = pd.read_csv('covid19_nlp_text_classification/Corona_NLP_test.csv')

In [ ]:
eval_data.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [ ]:
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'species'.
eval_data['Sentiment']= label_encoder.fit_transform(eval_data['Sentiment'])
 
eval_data['Sentiment'].unique()

array([0, 4, 1, 2, 3])

In [ ]:
eval_data.drop(columns = ['UserName','ScreenName','Location','TweetAt'], inplace=True)

In [ ]:
eval_data.rename(columns = {'OriginalTweet' : 'text' , 'Sentiment':'labels'})

,text,labels
0,TRENDING: New Yorkers encounter empty supermar...,0
1,When I couldn't find hand sanitizer at Fred Me...,4
2,Find out how you can protect yourself and love...,1
3,#Panic buying hits #NewYork City as anxious sh...,2
4,#toiletpaper #dunnypaper #coronavirus #coronav...,3
...,...,...
3793,Meanwhile In A Supermarket in Israel -- People...,4
3794,Did you panic buy a lot of non-perishable item...,2
3795,Asst Prof of Economics @cconces was on @NBCPhi...,3
3796,Gov need to do somethings instead of biar je r...,0


In [ ]:
eval_data.head()

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,0
1,When I couldn't find hand sanitizer at Fred Me...,4
2,Find out how you can protect yourself and love...,1
3,#Panic buying hits #NewYork City as anxious sh...,2
4,#toiletpaper #dunnypaper #coronavirus #coronav...,3


In [ ]:
# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1, overwrite_output_dir=True)

# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    'bert-base-cased',
    num_labels=5,
    args=model_args,
    use_cuda=False
) 

# Train the model
model.train_model(train_data)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/41157 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_5_2
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/5145 [00:00<?, ?it/s]

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_data)

In [ ]:
result

In [ ]:
model_outputs

In [ ]:
wrong_predictions

In [ ]:
# Make predictions with the model
predictions, raw_outputs = model.predict(["Sam was a Wizard"])

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
predictions

array([0])

In [ ]:
raw_outputs

array([[ 0.11218262,  0.00329399, -0.59619141]])